In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
import malaya

model = malaya.translation.ms_en.transformer(model = 'small')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [18]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
left = [unidecode(s) for s in ms]
right = [unidecode(s) for s in eng]

In [19]:
len(ms), len(eng)

(998, 998)

In [20]:
from tqdm import tqdm

batch_size = 16

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:08<00:00,  7.58it/s]


In [21]:
len(results)

998

In [22]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [23]:
refs = [filtered_right]
sys = filtered_left

In [24]:
r = bleu.corpus_score(sys, refs)

In [25]:
r.__dict__

{'name': 'BLEU',
 'score': 35.392560621251945,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '68.4/43.4/29.5/20.5 (BP = 0.967 ratio = 0.967 hyp_len = 22798 ref_len = 23570)',
 'bp': 0.966704301808668,
 'counts': [15597, 9461, 6145, 4058],
 'totals': [22798, 21801, 20804, 19807],
 'sys_len': 22798,
 'ref_len': 23570,
 'precisions': [68.41389595578559,
  43.397091876519426,
  29.53758892520669,
  20.487706366436107],
 'prec_str': '68.4/43.4/29.5/20.5',
 'ratio': 0.9672464997878659}

In [26]:
chrf.corpus_score(sys, refs)

chrF2++ = 59.64